### Importing the Necessary Libraries

In [1]:
import torch, torchvision
from torchvision import models, datasets, transforms
import torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os

print("Torch:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")


Torch: 2.4.1
MPS available: True


In [2]:
# STEP 2 — CIFAKE loader for structure: data/{train,test}/{FAKE,REAL}
import os, math, torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# --- config (matches your screenshot) ---
ROOT_DIR   = "data"          # contains 'train' and 'test'
IMG_SIZE   = 224
BATCH      = 64
FRACTION   = 0.10            # use only 10% per split for a quick run
VAL_RATIO  = 0.20            # build a stratified val from TRAIN
SEED       = 42
NUM_WORKERS = 2

g = torch.Generator().manual_seed(SEED)

tf_train = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
tf_eval = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

def stratified_split_indices(samples, num_classes, val_ratio, fraction, generator):
    per_class = {c: [] for c in range(num_classes)}
    for idx, (_, cls) in enumerate(samples):
        per_class[cls].append(idx)

    train_keep, val_keep = [], []
    for c, idxs in per_class.items():
        if not idxs: continue
        perm = torch.randperm(len(idxs), generator=generator).tolist()
        idxs = [idxs[i] for i in perm]
        k_frac = max(1, math.floor(len(idxs) * fraction))  # take only a fraction
        idxs = idxs[:k_frac]
        k_val = max(1, math.floor(len(idxs) * val_ratio))  # carve out val
        val_keep.extend(idxs[:k_val])
        train_keep.extend(idxs[k_val:])

    tperm = torch.randperm(len(train_keep), generator=generator).tolist()
    vperm = torch.randperm(len(val_keep),   generator=generator).tolist()
    train_keep = [train_keep[i] for i in tperm]
    val_keep   = [val_keep[i]   for i in vperm]
    return train_keep, val_keep

def stratified_subset_indices(samples, num_classes, fraction, generator):
    per_class = {c: [] for c in range(num_classes)}
    for idx, (_, cls) in enumerate(samples):
        per_class[cls].append(idx)
    keep = []
    for c, idxs in per_class.items():
        perm = torch.randperm(len(idxs), generator=generator).tolist()
        k = max(1, math.floor(len(idxs) * fraction))
        keep.extend([idxs[i] for i in perm[:k]])
    perm_all = torch.randperm(len(keep), generator=generator).tolist()
    return [keep[i] for i in perm_all]

# build datasets from your structure
train_full = datasets.ImageFolder(os.path.join(ROOT_DIR, "train"), transform=tf_train)
test_full  = datasets.ImageFolder(os.path.join(ROOT_DIR, "test"),  transform=tf_eval)

train_idx, val_idx = stratified_split_indices(
    train_full.samples, num_classes=len(train_full.classes),
    val_ratio=VAL_RATIO, fraction=FRACTION, generator=g
)
test_idx = stratified_subset_indices(test_full.samples, len(test_full.classes), FRACTION, g)

# wrap as subsets (val uses eval transforms)
val_full = datasets.ImageFolder(os.path.join(ROOT_DIR, "train"), transform=tf_eval)
train_ds = Subset(train_full, train_idx)
val_ds   = Subset(val_full,   val_idx)
test_ds  = Subset(test_full,  test_idx)

train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_dl  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print("classes:", train_full.classes, "(class_to_idx:", train_full.class_to_idx, ")")
print(f"FULL train:{len(train_full)}  FULL test:{len(test_full)}")
print(f"SUBSET train:{len(train_ds)}  val:{len(val_ds)}  test:{len(test_ds)}")


classes: ['FAKE', 'REAL'] (class_to_idx: {'FAKE': 0, 'REAL': 1} )
FULL train:100000  FULL test:20000
SUBSET train:8000  val:2000  test:2000


In [3]:
# STEP 3 — Minimal ResNet-18 train + validate on CIFAKE subset
import torch, torch.nn as nn, torch.optim as optim
from torchvision import models
from sklearn.metrics import accuracy_score, f1_score

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.set_float32_matmul_precision("high")

# build model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 2)   # binary classification
model.to(device)

# loss + optimizer
crit = nn.CrossEntropyLoss(label_smoothing=0.1)
opt  = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)

EPOCHS = 3  # short test run

for epoch in range(1, EPOCHS+1):
    # training
    model.train()
    for xb,yb in train_dl:
        xb,yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        with torch.autocast(device_type="mps", dtype=torch.float16, enabled=torch.backends.mps.is_available()):
            loss = crit(model(xb), yb)
        loss.backward()
        opt.step()
    print(f"epoch {epoch}: train loss {loss.item():.4f}")

    # validation
    model.eval()
    all_y, all_pred = [], []
    with torch.no_grad():
        for xb,yb in val_dl:
            xb = xb.to(device)
            with torch.autocast(device_type="mps", dtype=torch.float16, enabled=torch.backends.mps.is_available()):
                logits = model(xb)
            preds = logits.argmax(1).cpu()
            all_pred.extend(preds.numpy())
            all_y.extend(yb.numpy())
    acc = accuracy_score(all_y, all_pred)
    f1  = f1_score(all_y, all_pred)
    print(f"   val acc {acc:.3f}, f1 {f1:.3f}")


RuntimeError: User specified an unsupported autocast device_type 'mps'

In [1]:
import torch, torchvision
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("MPS available:", torch.backends.mps.is_available())


torch: 2.4.1
torchvision: 0.19.1
MPS available: True


In [2]:
from contextlib import nullcontext
import torch

def mps_autocast():
    # Only use autocast if (1) MPS is built, (2) MPS is available at runtime, and
    # (3) autocast("mps") actually works in this wheel.
    try:
        if getattr(torch.backends, "mps", None) and torch.backends.mps.is_built() and torch.backends.mps.is_available():
            # Probe once: if constructing the context fails, we fall back.
            _ = torch.autocast(device_type="mps", dtype=torch.float16)
            return torch.autocast(device_type="mps", dtype=torch.float16)
    except Exception:
        pass
    return nullcontext()


In [4]:
# =========================
# CIFAKE -> ResNet-18 (CPU)
# =========================

# -------- CONFIG --------
ROOT_DIR   = "data"   # contains 'train' and 'test' with subfolders FAKE/REAL
IMG_SIZE   = 224
BATCH      = 64
FRACTION   = 0.10     # use only 10% of each split to move fast; set to 1.0 for all data
VAL_RATIO  = 0.20     # from TRAIN (after fraction), carve out 20% for validation
EPOCHS     = 3
LR         = 3e-4
WEIGHT_DECAY = 5e-4
LABEL_SMOOTH = 0.1
SEED       = 42
NUM_WORKERS = 2
# ------------------------

import os, math, random, numpy as np
import torch, torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import torch.nn as nn, torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix

# Reproducibility
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Force CPU (avoids any MPS/autocast issues)
device = torch.device("cpu")
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("Using device:", device)

# --------- Transforms ----------
tf_train = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
tf_eval = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# --------- Helpers ----------
def stratified_split_indices(samples, num_classes, val_ratio, fraction, seed=SEED):
    g = torch.Generator().manual_seed(seed)
    per_class = {c: [] for c in range(num_classes)}
    for idx, (_, cls) in enumerate(samples):
        per_class[cls].append(idx)

    train_keep, val_keep = [], []
    for c, idxs in per_class.items():
        if not idxs: 
            continue
        perm = torch.randperm(len(idxs), generator=g).tolist()
        idxs = [idxs[i] for i in perm]
        k_frac = max(1, math.floor(len(idxs) * fraction))  # fraction first
        idxs = idxs[:k_frac]
        k_val = max(1, math.floor(len(idxs) * val_ratio))  # carve val
        val_keep.extend(idxs[:k_val])
        train_keep.extend(idxs[k_val:])
    # shuffle
    if train_keep:
        perm = torch.randperm(len(train_keep), generator=g).tolist()
        train_keep = [train_keep[i] for i in perm]
    if val_keep:
        perm = torch.randperm(len(val_keep), generator=g).tolist()
        val_keep = [val_keep[i] for i in perm]
    return train_keep, val_keep

def stratified_subset_indices(samples, num_classes, fraction, seed=SEED):
    g = torch.Generator().manual_seed(seed)
    per_class = {c: [] for c in range(num_classes)}
    for idx, (_, cls) in enumerate(samples):
        per_class[cls].append(idx)
    keep = []
    for c, idxs in per_class.items():
        if not idxs: continue
        perm = torch.randperm(len(idxs), generator=g).tolist()
        k = max(1, math.floor(len(idxs) * fraction))
        keep.extend([idxs[i] for i in perm[:k]])
    if keep:
        perm = torch.randperm(len(keep), generator=g).tolist()
        keep = [keep[i] for i in perm]
    return keep

# --------- Datasets / Loaders ----------
train_full = datasets.ImageFolder(os.path.join(ROOT_DIR, "train"), transform=tf_train)
test_full  = datasets.ImageFolder(os.path.join(ROOT_DIR, "test"),  transform=tf_eval)

train_idx, val_idx = stratified_split_indices(
    train_full.samples, num_classes=len(train_full.classes),
    val_ratio=VAL_RATIO, fraction=FRACTION, seed=SEED
)
test_idx = stratified_subset_indices(test_full.samples, len(test_full.classes), FRACTION, seed=SEED)

val_full = datasets.ImageFolder(os.path.join(ROOT_DIR, "train"), transform=tf_eval)
train_ds = Subset(train_full, train_idx)
val_ds   = Subset(val_full,   val_idx)
test_ds  = Subset(test_full,  test_idx)

train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=False)
val_dl   = DataLoader(val_ds,  batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS, pin_memory=False)
test_dl  = DataLoader(test_ds, batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS, pin_memory=False)

print("classes:", train_full.classes, "(class_to_idx:", train_full.class_to_idx, ")")
print(f"FULL train:{len(train_full)}  FULL test:{len(test_full)}")
print(f"SUBSET train:{len(train_ds)}  val:{len(val_ds)}  test:{len(test_ds)}")

# Identify which class index to treat as "positive" for AUROC
# Here we'll use 'FAKE' as positive if it exists, else class 1.
cls_to_idx = train_full.class_to_idx
pos_idx = cls_to_idx.get("FAKE", 1)

# --------- Model / Optim ---------
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 2)
model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# --------- Train + Val ----------
for epoch in range(1, EPOCHS+1):
    model.train()
    running_loss = 0.0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)          # FP32 CPU
        loss   = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)
    train_loss = running_loss / len(train_dl.dataset)

    # validation
    model.eval()
    y_true, y_pred = [], []
    y_prob_pos = []
    with torch.no_grad():
        for xb, yb in val_dl:
            xb = xb.to(device)
            logits = model(xb)
            probs  = torch.softmax(logits, dim=1)
            preds  = logits.argmax(1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(yb.numpy())
            y_prob_pos.extend(probs[:, pos_idx].cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred)
    try:
        auroc = roc_auc_score(y_true, np.array(y_prob_pos))
    except Exception:
        auroc = float("nan")

    print(f"Epoch {epoch:02d} | train loss {train_loss:.4f} | val Acc {acc:.3f} F1 {f1:.3f} AUROC {auroc:.3f}")

# --------- Test Evaluation ----------
model.eval()
y_true, y_pred, y_prob_pos = [], [], []
with torch.no_grad():
    for xb, yb in test_dl:
        xb = xb.to(device)
        logits = model(xb)
        probs  = torch.softmax(logits, dim=1)
        preds  = logits.argmax(1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(yb.numpy())
        y_prob_pos.extend(probs[:, pos_idx].cpu().numpy())

test_acc = accuracy_score(y_true, y_pred)
test_f1  = f1_score(y_true, y_pred)
try:
    test_auroc = roc_auc_score(y_true, np.array(y_prob_pos))
except Exception:
    test_auroc = float("nan")
cm = confusion_matrix(y_true, y_pred)

print("\n=== TEST RESULTS ===")
print(f"Accuracy: {test_acc:.4f}  F1: {test_f1:.4f}  AUROC: {test_auroc:.4f}")
print("Confusion matrix (rows=true, cols=pred):\n", cm)


torch: 2.4.1
torchvision: 0.19.1
Using device: cpu
classes: ['FAKE', 'REAL'] (class_to_idx: {'FAKE': 0, 'REAL': 1} )
FULL train:100000  FULL test:20000
SUBSET train:8000  val:2000  test:2000
Epoch 01 | train loss 0.3811 | val Acc 0.896 F1 0.886 AUROC 0.036


KeyboardInterrupt: 